## Yeji's Code

In [22]:
import pandas as pd

import mglearn
import random
import numpy as np

def train_test_split(X, y, test_size=0.25, random_state=None):
    
    # Set random seed for reproducibility if random_state is provided
    if random_state is not None:
        np.random.seed(random_state)
        
        
    # Get the total number of samples
    n_samples = len(X)
    
    
    # Create an array of indices and shuffle them
    indices = np.arange(n_samples)
    np.random.shuffle(indices)

    # Determine the number of samples for the test set
    if isinstance(test_size, float):
        test_size = int(test_size * n_samples)
        
        
    # Extract indices for the test and training sets
    test_indices = indices[:test_size]
    train_indices = indices[test_size:]
    
    
    # Use indices to split the data into training and testing sets
    X_train, X_test = X[train_indices], X[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    return X_train, X_test, y_train, y_test

def fit_linear_regression(X, y):
    
    # Add a column of ones to X for the intercept term
    X_ext = np.column_stack((np.ones(len(X)), X))
    
    # Calculate coefficients using the normal equation
    coefficients = np.dot(np.dot(np.linalg.pinv(np.dot(X_ext.T, X_ext)), X_ext.T), y)
    
    return coefficients

def predict(X, coefficients):
    
    
    # Add a column of ones to X for the intercept term
    X_ext = np.column_stack((np.ones(len(X)), X))
    
    # Calculate predicted target values using dot product
    y_pred = np.dot(X_ext, coefficients)
    
    return y_pred
    
def mean_squared_error(y_true, y_pred):
    
    
    # Ensure that the input arrays have the same length
    if len(y_true) != len(y_pred):
        raise ValueError("Input arrays must have the same length.")

    # Calculate squared differences
    squared_diff = [(true - pred) ** 2 for true, pred in zip(y_true, y_pred)]

    # Calculate mean squared error
    mse = sum(squared_diff) / len(y_true)

    return mse
# Load the extended Boston Housing dataset
X, y = mglearn.datasets.load_extended_boston()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




# Fit the linear regression model on the training set
coefficients = fit_linear_regression(X_train, y_train)

# Make predictions on the training set
y_train_pred = predict(X_train, coefficients)

# Make predictions on the test set
y_pred = predict(X_test, coefficients)


# Calculate Mean Squared Error (MSE) on the test set
mse_test = mean_squared_error(y_test, y_pred)


# Calculate Mean Squared Error (MSE) on the training set
mse_train = mean_squared_error(y_train, y_train_pred)
print("Mean Squared Error on Training Set:", mse_train)
print("Mean Squared Error on Test Set:", mse_test) 

Mean Squared Error on Training Set: 5.119969179921979
Mean Squared Error on Test Set: 14.329434191662344


## Ridge Regression

Ridge regression, also known as Tikhonov regularization or L2 regularization, takes the least squares function and adds a regularization term to it. This is typically used to prevent overfitting.

The ridge regression function is given by:

$$
\text{J(θ)} = MSE(θ) + α\sum_{i=1}^{n} (θ_i)^2 
$$

Where:
- $\text{J(θ)}$ is the cost function to be minimized.
- $MSE(θ)$ is the Mean Squared Error term.
- $α$ is the regularization parameter.
- $θ_i$ are the regression coefficients.

The ridge regression coefficients are obtained by minimizing this cost function:

$$
\hat{θ} = \text{argmin}_θ J(θ)
$$

The closed-form solution for ridge regression is given by:

$$
\hat{θ} = (X^T X + αI)^{-1} X^T y
$$

Here:
- $\hat{θ}$ is the vector of ridge regression coefficients.
- $X$ is the matrix of input features.
- $y$ is the vector of target values.
- $I$ is the identity matrix.

Iterative methods like gradient descent are also used for ridge regression.

In [23]:
import numpy as np

# closed-form approach
def RidgeRegression(X, Y, alpha):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    I = np.identity(X_b.shape[1])
    I[0, 0] = 0
    
    theta = np.linalg.inv(X_b.T.dot(X_b) + alpha * I).dot(X_b.T).dot(Y)
    
    # Separate bias and weights
    w = theta[1:]
    b = theta[0]

    return w, b

# gradient descent approach
def gdRR(X, y, alpha, lr, n_iterations):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    theta = np.zeros(X_b.shape[1])

    for i in range(n_iterations):
        pred = np.dot(X_b, theta)
        errors = y - pred
        gradients = (-2/len(y)) * np.dot(X_b.T, errors) + alpha * theta
        theta -= lr * gradients

    # Separate bias and weights
    w = theta[1:]
    b = theta[0]

    return w, b

In [58]:
test_alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
results = []
w_set = []
b_set = []

results1 = []
w_set1 = []
b_set1 = []

for alpha in test_alphas:
  w, b = RidgeRegression(X_train, y_train, alpha)
  Y_pred = np.dot(X_test, w) + b

  results.append(mean_squared_error(y_test, Y_pred))
  w_set.append(w)
  b_set.append(b)

w_set = [w[0] for w in w_set]

for alpha in test_alphas:
  w1, b1 = gdRR(X_train, y_train, alpha, lr = 0.001, n_iterations = 1000)
  Y_pred1 = np.dot(X_test, w1) + b1

  results1.append(mean_squared_error(y_test, Y_pred1))
  w_set1.append(w1)
  b_set1.append(b1)

w_set1 = [w1[0] for w1 in w_set1]

data = {"Alpha": test_alphas, "w": w_set, "b": b_set, "MSE": results}
df = pd.DataFrame(data)
display(df)

gd_data = {"Alpha": test_alphas, "w": w_set1, "b": b_set1, "MSE": results1}
gd_df = pd.DataFrame(gd_data)
gd_df.astype(int)
display(gd_df)


,Alpha,w,b,MSE
0,0.001,5.288569,-25.050419,13.519711
1,0.010,-5.665918,-14.889592,12.257621
2,0.100,-3.472767,6.438892,11.013404
3,1.000,-1.493739,19.766685,12.710819
4,10.000,-0.856798,23.426068,20.179910
5,100.000,-0.293358,25.241065,33.025438
6,1000.000,-0.086045,24.776840,53.970180


,Alpha,w,b,MSE
0,0.001,-0.247627,4.967628,48.727234
1,0.010,-0.246692,4.957781,48.779237
2,0.100,-0.237575,4.861728,49.366433
3,1.000,-0.166187,4.098162,59.345162
4,10.000,-0.006244,1.934494,158.980835
5,100.000,0.008228,0.394774,416.922203
6,1000.000,0.001101,0.044879,521.013811


From the above table, it can be seen that the lowest MSE for this dataset is when α = 0.1, which is our optimal regularization.

The ridge regression function can be found from sklearn.linear_model.

In [59]:
from sklearn.linear_model import Ridge

for alpha in test_alphas:
    ridge_model = Ridge(alpha=alpha)

    # Train the model
    ridge_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = ridge_model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")

Mean Squared Error: 13.519710896811109
Mean Squared Error: 12.257621202214008
Mean Squared Error: 11.013404437259284
Mean Squared Error: 12.710819494074434
Mean Squared Error: 20.17990981392928
Mean Squared Error: 33.02543777154453
Mean Squared Error: 53.970179940917824


As you can see, the MSE of the different alpha levels match.

## LASSO Regression

LASSO stands for Least Absolute Shrinkage and Selection Operator, is a linear regression technique with added regularization. It combines the simplicity of linear regression with the regularization power of the L1 norm to prevent overfitting and perform feature selection.

The cost function of LASSO regression is given by:

$$
J(\theta) = MSE(\theta) + α\sum_{i=1}^{n} |\theta_i|^2 
$$

Where:
- $\text{J(θ)}$ is the cost function to be minimized.
- $MSE(θ)$ is the Mean Squared Error term.
- $α$ is the regularization parameter.
- $θ_i$ are the regression coefficients.

The ridge regression coefficients are obtained by minimizing this cost function:

$$
\hat{\theta} = \text{argmin}_\theta J(\theta)
$$

There is no general closed-form solution for LASSO regression but is commonly minimized through gradient descent.

Our cost function can also be written as:

$$
J(\hat{\theta})=\frac{1}{2m} \big|\big| Y-\hat{\theta}X \big|\big| + λ∑_{i=1}^{n} \hat{|\theta_i|}
$$

We are looking to minimize the cost fuction with respect to W and b
Where $\hat{\theta} = \begin{bmatrix}  b & {\hat{\theta_1}} &.&.&.& {\hat{\theta_n}} \end{bmatrix}$
and $W = \begin{bmatrix} \hat{\theta}_1 &.&.&.& \hat{\theta_n} \end{bmatrix}$

so $\frac{d}{dW}J(\hat{\theta}) = \frac{1}{m}X^T(Y-\hat{\theta}X) + sgn(λ)$

and $\frac{d}{dW}J(\hat{\theta}) = \frac{1}{m}(Y-\hat{\theta}X)$

Then using the gradient descent:
$$
\theta_{k+1}=\theta_{k}-h \nabla J\left(\theta_{k}\right).
$$

Where $\theta_{k}$ is our initial guess and $\theta_{k+1}$ is our new guess and $h$ is our step size

In [60]:
def lasso_regression(X, y, alpha, num_iterations, learning_rate):
    
    # Get m the number of observations and n the number of independent variables
    m, n = X.shape 
    
    # Initialize weights
    W = np.zeros(n) 

    # Initialize intercept term
    b = 0
    
    # Performing gradient descent to find the minimum of out cost function
    for iteration in range(num_iterations):
        predictions = np.dot(X, W) + b
        residuals = predictions - y
        dJdW = (1/m)*np.dot(X.T, residuals) + alpha * np.sign(W)
        dJdb = (1/m)*np.sum(residuals)
        
        W -= learning_rate * dJdW
        b -= learning_rate * dJdb

    return W, b


In [77]:
las_results = []

for alpha in test_alphas:
  w, b = lasso_regression(X_train, y_train, alpha,num_iterations = 10000, learning_rate = 0.05)
  Y_pred = np.dot(X_test, w) + b

  las_results.append(mean_squared_error(y_test, Y_pred))

print(las_results)


[11.709476021279247, 12.98760660075295, 23.336703060799305, 55.15641668621716, 79.85411975941231, 95.95483057535972, 537283.3973995898]


In [72]:
from sklearn.linear_model import Lasso

for alpha in test_alphas:
    lasso_model = Lasso(alpha=alpha,max_iter = 100000)

    # Train the model
    lasso_model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = lasso_model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse}")

Mean Squared Error: 11.413035283805366
Mean Squared Error: 12.901685764529885
Mean Squared Error: 22.456548850036334
Mean Squared Error: 55.31351982194934
Mean Squared Error: 75.76013971789499
Mean Squared Error: 75.76013971789499
Mean Squared Error: 75.76013971789499


We can see that up to alpha = 10 the MSE of our gradient descent is approximately equal to the sklearn LASSO MSE. This will differ by changing up the number of iterations and learning rate.

Also alpha = 0.001 or 0.01 seems to minimize the MSE

In [100]:
w, b = lasso_regression(X_train, y_train, alpha=0.01,num_iterations = 20000, learning_rate = 0.2)

print(w)
print(b)

[-2.79832438e+00 -7.47020560e-04  1.46094759e-03  3.72275914e-03
  2.06159658e-03  6.16281787e+00  2.06069301e-05 -7.02924048e+00
  1.16597680e+01  9.12567569e-03  1.00474192e-02  2.95767832e+00
 -3.67698944e-01 -1.47728219e-03 -1.30515945e-03  6.41512417e-04
 -8.92262371e-04 -1.55593161e-04 -1.57692283e-03 -3.44169261e-03
  6.34993616e-04 -5.20725675e+00 -2.01929127e-01 -4.80841776e-05
 -3.78587849e-04 -3.33903155e-04  1.59696255e+00 -1.29441540e-04
  3.16875378e-04 -1.01312877e-03  1.63510406e-03  1.19313114e-03
  2.53004382e-03  3.25146953e-04  5.00594370e-04  1.71481584e-03
  1.41086071e-03  4.80604119e-04  6.63936913e-01  1.30110874e-03
  5.50393393e-03  5.17308873e-04  5.20956213e-03 -1.17267951e+00
  3.28504886e-03  7.80133876e+00 -3.41686199e-03  7.11978731e-03
 -2.10009018e+00  3.72275914e-03 -5.32177617e+00 -1.85001077e+00
  3.83464107e-03 -2.02891418e-03  1.33952997e+00  7.14997392e+00
  9.97250881e-04  1.87611370e+00 -7.02199565e-04 -2.46696920e+00
 -3.57392990e+00 -1.36082

In [91]:
# Create a Lasso regression model
lasso_model = Lasso(alpha=0.01,max_iter = 100000)
# Fit the model to the training data
lasso_model.fit(X_train, y_train)
# Make predictions on the test data
y_pred = lasso_model.predict(X_test)

print(lasso_model.coef_)
print(lasso_model.intercept_)

[-0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -0.00000000e+00 -8.61448752e+00
  1.32747817e+01  0.00000000e+00 -0.00000000e+00  4.22948747e+00
 -6.79244783e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -8.10113946e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  1.80657247e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  7.84771010e-03  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -8.13870499e-01
  0.00000000e+00  7.87205046e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000000e+00 -6.47720077e+00 -2.21214790e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  9.61987786e+00
  0.00000000e+00  2.20457393e+00 -0.00000000e+00 -2.49419166e+00
 -3.13735059e+00 -0.00000

If we compare the coefficients of sklearn LASSO to the gradient descent, the gradient descent has almost the same values as the sklearn one

But notice that we had to change the number of iterations and learning rate to find a close approximate to the sklearn model

In [101]:
Y_pred = np.dot(X_test, w) + b
print(mean_squared_error(y_test, Y_pred))

12.295366661959434


The MSE now for our LASSO function is 12.295366661959434 now further from the SKlearn MSE of 12.901685764529885 but the coeffcients are now closer.